In [3]:
!pip install Pillow

In [9]:
import numpy as np
import pandas as pd
from PIL import Image
import glob
import os
import shutil 
import matplotlib.pyplot as plt

In [14]:
VERSION = 1
n_sessions = 5
n_images_per_session = 400
target_images_dirpath = '/Users/aidapiccato/PycharmProjects/concentration/concentration-game-mworks/images/v%s/' % str(VERSION)

In [37]:
source_dir = '/Users/aidapiccato/lamem/images/'
train_filenames = glob.glob('/Users/aidapiccato/lamem/splits/train_*')
print(train_filenames)

['/Users/aidapiccato/lamem/splits/train_4.txt', '/Users/aidapiccato/lamem/splits/train_5.txt', '/Users/aidapiccato/lamem/splits/train_1.txt', '/Users/aidapiccato/lamem/splits/train_2.txt', '/Users/aidapiccato/lamem/splits/train_3.txt']


Aggregating memorability score across training splits

In [27]:
fn = train_filenames[2]
a = np.asarray([[x.split(' ')[0], np.float(x.split(' ')[1][:-1])] for x in open(fn).readlines() for fn in train_filenames])
df = pd.DataFrame(dict(zip(['image_name', 'mem_score'], a.transpose())))
df['mem_score'] = df['mem_score'].apply(np.float)
df = df.groupby('image_name').agg(np.mean)

Selecting images whose memorability scores falls within a certain range

In [96]:
low_mem, hi_mem = 0.7, 0.74
valid = df[df.mem_score.between(low_mem, hi_mem)]
valid.reset_index(inplace=True)

image_names = valid['image_name'].values
np.random.seed(VERSION)
np.random.shuffle(image_names)
image_names = image_names[:((n_sessions + 1) * n_images_per_session)]
version_dir = '/Users/aidapiccato/PycharmProjects/concentration/concentration-game-mworks/images/v%d/all_images/' % VERSION
try:
    os.mkdir(version_dir)
except:
    shutil.rmtree(version_dir)
    os.mkdir(version_dir)
valid_image_names = []
for image_fn in image_names:
    try:
        shutil.copy(source_dir + image_fn, version_dir)
        valid_image_names.append(image_fn)
    except:
        pass
valid_image_names = np.asarray(valid_image_names)
len(valid_image_names)

Creating directory for version

Creating subdirectories for each session 

In [99]:
def create_session_dir(subject_id, sess_index, image_names, n_sessions=5):
    seed = hash(subject_id) % (2**32 - 1)
    np.random.seed(seed)
    np.random.shuffle(image_names)
    split_image_names = np.split(image_names[0:len(image_names) - len(image_names)%n_sessions], n_sessions)
    desired_size = 368
    target_dir = '/Users/aidapiccato/PycharmProjects/concentration/concentration-game-mworks/curr_images/'
    try:
        os.mkdir(target_dir)
    except:
        shutil.rmtree(target_dir)
        os.mkdir(target_dir)
    sess_image_filenames = split_image_names[sess_index][:n_images_per_session]
    print(len(sess_image_filenames))
    for image_fn in sess_image_filenames:
        image = Image.open(version_dir + image_fn)
        old_size = image.size
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        image = image.resize(new_size, Image.ANTIALIAS)
        new_image = Image.new("RGB", (desired_size, desired_size))
        new_image.paste(image, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))
        new_image.save(target_dir + image_fn)
create_session_dir(0, 0, valid_image_names)

400


In [90]:
target_dir = '/Users/aidapiccato/PycharmProjects/concentration/concentration-game-mworks/curr_images/*'
len(glob.glob(target_dir))

400